In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install ibm-cos-sdk

In [ ]:
#connect to Cloud Object Storage
import ibm_boto3
from ibm_botocore.client import Config, ClientError

# Constants for IBM COS values
COS_ENDPOINT = "https://s3.us-south.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

COS_API_KEY_ID = "w9OYjZ0qricse-Z0Sar08ccWq6XQw_3HmG4uUzA6qXwm" # eg "W00YixxxxxxxxxxMB-odB-2ySfTrFBIQQWanc--P3byk"

COS_INSTANCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/a2fbcfc73145484da67dd67dbc4f914d:c9ef8a5f-f92d-4fd7-a5a8-8cb94bb46b28::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003xxxxxxxxxx1c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"

# Create client 
cos = ibm_boto3.client("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_INSTANCE_CRN,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [ ]:
package.info(), component.info(), mastertable.info(), model.info(), derive.info()

In [11]:
data = pd.read_csv(cos.get_object(Bucket='test-wml-flow', Key='DATA_3506-CX_-001.csv')['Body'])
#data = pd.read_csv(r'D:\Alpha Com\New Platform\TEST_WML_FLOW\DATA_3506-CX_-001.csv')

In [12]:
def upload_data(data,name):
    training_data = data
    model_name = name
    cos.put_object(Body=training_data.to_csv(encoding='utf-8', index=False), Bucket='training-model-data', Key=model_name+'.csv')

In [ ]:
package_table = cos.get_object(Bucket='test-wml-flow', Key='PACKAGES.csv')['Body']
package = pd.read_csv(package_table)

component_table = cos.get_object(Bucket='test-wml-flow', Key='COMPONENT.csv')['Body']
component = pd.read_csv(component_table)

master_table = cos.get_object(Bucket='test-wml-flow', Key='MASTER_TABLE.csv')['Body']
mastertable = pd.read_csv(master_table)

model_table = cos.get_object(Bucket='test-wml-flow', Key='MODEL.csv')['Body']
model = pd.read_csv(model_table)

derive_table = cos.get_object(Bucket='test-wml-flow', Key='DERIVE_TARGET.csv')['Body']
derive = pd.read_csv(derive_table)

training_table = cos.get_object(Bucket='test-wml-flow', Key='TRAINING_DATA_LIST.csv')['Body']
training = pd.read_csv(training_table)

def upload_data(data,name):
    training_data = data
    model_name = name
    cos.put_object(Body=training_data.to_csv(encoding='utf-8', index=False), Bucket='training-model-data', Key=model_name+'.csv')

def data_for_training(inputpackage):
    package_name = inputpackage
    df1 = package[package['PACKAGE_NAME']==package_name]
    package_id = df1['PACKAGE_ID'].iloc[0]

    df2 = component[component['PACKAGE_ID']==package_id]
    component_list = df2['COMPONENT_ID'].values

    cols = ['OPC_CREATED_DT']
    data_timestamp = data[cols]  

    model_input = model
    #model_input['INPUT_TRAINING_NAME'] = ''

   
    for i in component_list:
        component_id = i
        df3 = component[component['COMPONENT_ID']==component_id]
        component_name = str(df3['COMPONENT_NAME'].iloc[0])

        df4 = model[model['COMPONENT_ID'] == component_id]
        model_list = df4['MODEL_ID'].values
        derive_list = derive['MODEL_ID'].values
            
        for model_id in model_list:
            #model_id_loc = int(model_id)-1
            model_name = package_name+'_'+component_name+'_'+str(model_id)
            print(model_id, model_name)

            df5 = mastertable[mastertable['MODEL_ID'] == model_id]
            target = df5[df5['TYPE'] =='TARGET']
            predictor = df5[df5['TYPE'] =='PREDICTOR']
           
            targetlist = list(target['SENSOR_ID'].values)
            predictorlist = list(predictor['SENSOR_ID'].values)
            date = list(['OPC_CREATED_DT'])
            cols = date + predictorlist + targetlist
            print(cols)
            training_data = data[cols]
            #print(training_data.info())

            #check if 
            if model_id in derive_list:
                df6 = derive[derive['MODEL_ID']==model_id]
                cal = df6['CALCULATE'].values
            
                if cal == 'AVG':
                    training_data['sum_num'] = 0
                    derivename = 'AVG'
                    for n in targetlist:
                        n = n.replace("PTTGSP.GSP5.3506.3506CX01_SALES GAS COMP.", "")
                        derivename = derivename+'_'+n
                    for i in targetlist:
                        training_data['sum_num'] = training_data['sum_num'] + training_data[i]   

                    training_data[derivename] = training_data['sum_num']/len(targetlist)
                    cols_drop = list(['sum_num']) + list(targetlist)
                    training_data = training_data.drop(columns=cols_drop)
                    training_data.info()
                    upload_data(training_data,model_name)
                    model_input.loc[(model_input.MODEL_ID == model_id ),'INPUT_TRAINING_NAME'] = model_name
                    print(derivename)
                    print('-------------------------AVG----------------------------')
                
                if cal == 'DIFF':
                    training_data['diff_num'] = 0
                    derivename = 'DIFF'
                    for n in targetlist:
                        n = n.replace("PTTGSP.GSP5.3506.3506CX01_SALES GAS COMP.", "")
                        derivename = derivename+'_'+n
                    for i in targetlist:
                        training_data['diff_num'] = training_data[i] - training_data['diff_num']

                    training_data[derivename] = training_data['diff_num'].abs()
                    cols_drop = list(['diff_num']) + list(targetlist)
                    training_data = training_data.drop(columns=cols_drop)
                    training_data.info()
                    upload_data(training_data,model_name)
                    model_input.loc[(model_input.MODEL_ID == model_id ),'INPUT_TRAINING_NAME']= model_name
                    print(derivename)
                    print('-------------------------DIFF----------------------------')

            #if no derive column
            else:
                training_data.info()
                upload_data(training_data,model_name)
                model_input.loc[(model_input.MODEL_ID == model_id ),'INPUT_TRAINING_NAME']= model_name
                print('-------------------------NORMAL----------------------------')
        
    print(model_input)
    cos.put_object(Body=model_input.to_csv(encoding='utf-8', index=False), Bucket='test-wml-flow', Key='MODEL'+'.csv')

           
     

In [ ]:
data_for_training('3506-CX-001')

In [ ]:
print('prepare training input data for each model')